# Title

In [ ]:
from icalendar import Calendar, Event
from datetime import datetime
import time as Time
import uuid

calendar = {}
prodid = "1" #TODO: change this to our name

def to_ics(response):
	input = response.split(",")
	date = input[0]
	time = input[1]
	summary = input[2]
	date = date.split("/")
	endtime=""
	if "全天" in time:
		time = "00:00~23:59"
	if "~" in time:
		time = time.split("~")
		endtime= time[1]
		time = time[0]
	time = time.split(":")
	if endtime!="":
		endtime = endtime.split(":")

	cal = Calendar()
	cal.add('version', '2.0')
	cal.add('prodid', prodid)

	event = Event()

	now_time = datetime(
    Time.localtime()[0],
    Time.localtime()[1],
    Time.localtime()[2],
    Time.localtime()[3],
    Time.localtime()[4],
    Time.localtime()[5])

	event.add('summary', summary)
	event.add('dtstart', datetime(int(date[0]), int(date[1]), int(date[2]), int(time[0]), int(time[1])))
	if endtime!="":
		event.add('dtend', datetime(int(date[0]), int(date[1]), int(date[2]), int(endtime[0]), int(endtime[1])))
	else:
		event.add('dtend', datetime(int(date[0]), int(date[1]), int(date[2]), int(time[0]), int(time[1])))
	event.add('dtstamp', now_time)
	uid = uuid.uuid1()
	event['uid'] = uid
	cal.add_component(event)

	calendar[summary] = (summary, time, endtime, cal.to_ical().decode('utf-8'))
	f = open(f'{uid}.ics', 'wb')
	f.write(cal.to_ical())
	f.close()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

device = "cuda" # the device to load the model onto

torch.cuda.empty_cache()

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="balanced_low_0"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")
streamer = TextStreamer(tokenizer, skip_prompt = True, skip_special_tokens = True)

In [ ]:
time = str(Time.localtime()[0])+'/'+str(Time.localtime()[1])+'/'+str(Time.localtime()[2])
messages = [
	{
		"role": "system", 
		"content": f"""今天的日期是 {time} ，现在请说中文。user会给你一些文字，请你从文字中提取信息，告诉user在这些文字中交代了在什么时间要做什么事情。以这样的格式回答：
	日期, 时间, 事件
	要求：日期格式为 YYYY/MM/DD ，时间格式为 HH:mm 。如果时间是 12 小时制，转换为 24 小时制。事件请为其总结名称。如果没有提到时间，时间填写 全天 。仅当提供了开始时间和结束时间（或者可由时长计算）时，时间格式为 HH:mm ~ HH:mm，代表开始时间、结束时间，否则仅以 HH:mm 格式回答开始时间。如果没有交代任何事件，请不要回答任何内容。
	"""
	}
]

def send(prompt):
	messages.append({"role": "user", "content": prompt})
	text = tokenizer.apply_chat_template(
		messages,
		tokenize=False,
		add_generation_prompt=True
	)
	model_inputs = tokenizer([text], return_tensors="pt").to(device)

	generated_ids = model.generate(
		**model_inputs,
		max_new_tokens = 512,
		streamer = streamer
	)
	generated_ids = [
		output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
	]

	response = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)[0]
	messages.append({"role": "assistant", "content": response})

	return response

while True:
	prompt = input("user:")
	print(prompt)
	if prompt != "":
		response = send(prompt).replace("，", ",")
		to_ics(response)
	else:
		break

## TODO List

前端

导入日历

获取信息